In [10]:
import prepare as p
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn import metrics

import itertools



Fit the random forests classifier to your training sample and transform (i.e. make predictions on the training sample)

Use the titanic data

In [3]:
titanic = p.prep_titanic()
print(titanic.shape)
print(titanic['survived'].mean())
titanic.head(3)

(891, 15)
0.3838383838383838


,passenger_id,survived,sex,age,sibsp,parch,fare,class,embark_town,alone,embark_town_queenstown,embark_town_southampton,class_second,class_third,sex_male
0,0,0,male,22.0,1,0,7.2500,Third,Southampton,0,0,1,0,1,1
1,1,1,female,38.0,1,0,71.2833,First,Cherbourg,0,0,0,0,0,0
2,2,1,female,26.0,0,0,7.9250,Third,Southampton,1,0,1,0,1,0


Split data 70/20/10

In [4]:
train, validate, test = p.split_data(df=titanic, test_size=.1, validate_size=.2, 
                                     stratify_col='survived', random_state=25)
print(len(train), len(validate), len(test))
print(train['survived'].mean(), validate['survived'].mean(), test['survived'].mean())

623 178 90
0.38362760834670945 0.38202247191011235 0.3888888888888889


split X features and y target

In [5]:
def x_y_split(data_set, target, features):
    X = data_set[features]
    y = data_set[target]
    return X, y

Get desired columns

In [6]:
titanic.columns

Index(['passenger_id', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'class', 'embark_town', 'alone', 'embark_town_queenstown',
       'embark_town_southampton', 'class_second', 'class_third', 'sex_male'],
      dtype='object')

In [7]:
features = ['sibsp', 'parch', 'fare', 'alone', 'embark_town_queenstown', 
            'embark_town_southampton', 'class_second', 'class_third', 'sex_male']

X_train, y_train = x_y_split(data_set=train, target='survived', features=features)

X_validate, y_validate = x_y_split(data_set=validate, target='survived', features=features)

X_test, y_test = x_y_split(data_set=test, target='survived', features=features)

# verify
print(len(X_train),len(X_validate),len(X_test))

623 178 90


run baseline model

In [25]:
def run_baseline_model(train, validate, target):
    
    # Make Dummy classifier 
    clf = DummyClassifier(strategy="most_frequent")
    
    # Fit Dummy classifier
    clf.fit(X_train, y_train)
    
    # Predict
    y_pred_train = clf.predict(X_train)
    y_pred_val = clf.predict(X_validate)

    scores = []

    output = {
    'features': features,
    'model': 'baseline',
    'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
    'train_precision': metrics.precision_score(y_train, y_pred_train),
    'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
    'train_f1': metrics.f1_score(y_train, y_pred_train),
    'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
    'validate_precision': metrics.precision_score(y_validate, y_pred_val),
    'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
    'validate_f1': metrics.f1_score(y_validate, y_pred_val),
    }
    scores.append(output)
    
    scores_df = pd.DataFrame(scores)
    
    return scores_df

In [26]:
display(run_baseline_model(train, validate, 'survived'))

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,features,model,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
0,"[sibsp, parch, fare, alone, embark_town_queens...",baseline,0.616372,0.0,0.0,0.0,0.617978,0.0,0.0,0.0


**1.**
- **What is your baseline prediction?** 0, did not survive
- **What is your baseline accuracy?** 61.2%

**2.**

- **Fit the decision tree classifier to your training sample and transform (make predictions)**

Make model

In [62]:
clf = LogisticRegression(C=1, max_iter=150, random_state=123)


Fit model

In [63]:
clf.fit(X_train, y_train)

LogisticRegression(C=1, max_iter=150, random_state=123)

make predicctions

In [64]:
y_pred_train = clf.predict(X_train)
y_pred_val = clf.predict(X_validate)

2. Evaluate your results using the model score, confusion matrix, and classification report.

In [65]:
print(f'Model score on training set: {clf.score(X_train, y_train)}')
print(f'Confusion Matrix on training set:\n {metrics.confusion_matrix(y_train, y_pred_train)}')
print(f'Classification Report on training set:\n {metrics.classification_report(y_train, y_pred_train)}')

Model score on training set: 0.8089887640449438
Confusion Matrix on training set:
 [[336  48]
 [ 71 168]]
Classification Report on training set:
               precision    recall  f1-score   support

           0       0.83      0.88      0.85       384
           1       0.78      0.70      0.74       239

    accuracy                           0.81       623
   macro avg       0.80      0.79      0.79       623
weighted avg       0.81      0.81      0.81       623



In [66]:
print(f'Model score on validation set: {clf.score(X_validate, y_validate)}')
print(f'Confusion Matrix on validation set:\n {metrics.confusion_matrix(y_validate, y_pred_val)}')
print(f'Classification Report on validation set:\n {metrics.classification_report(y_validate, y_pred_val)}')

Model score on validation set: 0.7528089887640449
Confusion Matrix on validation set:
 [[91 19]
 [25 43]]
Classification Report on validation set:
               precision    recall  f1-score   support

           0       0.78      0.83      0.81       110
           1       0.69      0.63      0.66        68

    accuracy                           0.75       178
   macro avg       0.74      0.73      0.73       178
weighted avg       0.75      0.75      0.75       178



Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

Run through steps 2-4 using a different `max_depth` value.



In [74]:
list(range(-3,4))

[-3, -2, -1, 0, 1, 2, 3]

In [78]:
scores = []

for c in [1*10**x for x in range(-3, 4)]:
    # Make KNN classifier 
    clf = LogisticRegression(C=c, max_iter=150, random_state=123)

    # Fit KNN classifier
    clf.fit(X_train, y_train)
    
    # Predict
    y_pred_train = clf.predict(X_train)
    y_pred_val = clf.predict(X_validate)
    
    output = {
        'model': f'LogReg: C={c}',
        'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
        'train_precision': metrics.precision_score(y_train, y_pred_train),
        'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
        'train_f1': metrics.f1_score(y_train, y_pred_train),
        'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
        'validate_precision': metrics.precision_score(y_validate, y_pred_val),
        'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
        'validate_f1': metrics.f1_score(y_validate, y_pred_val),
    }

    scores.append(output)
    

pd.DataFrame(scores)

,model,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
0,LogReg: max_depth=0.001,0.670947,0.707317,0.242678,0.361371,0.713483,0.840000,0.308824,0.451613
1,LogReg: max_depth=0.01,0.731942,0.805085,0.397490,0.532213,0.730337,0.857143,0.352941,0.500000
2,LogReg: max_depth=0.1,0.812199,0.808081,0.669456,0.732265,0.741573,0.696429,0.573529,0.629032
3,LogReg: max_depth=1,0.808989,0.777778,0.702929,0.738462,0.752809,0.693548,0.632353,0.661538
4,LogReg: max_depth=10,0.807384,0.774194,0.702929,0.736842,0.752809,0.693548,0.632353,0.661538
5,LogReg: max_depth=100,0.807384,0.774194,0.702929,0.736842,0.752809,0.693548,0.632353,0.661538
6,LogReg: max_depth=1000,0.807384,0.774194,0.702929,0.736842,0.752809,0.693548,0.632353,0.661538


Which model performs better on your in-sample data?

Which model performs best on your out-of-sample data, the validate set?

In [80]:
strong_model_columns = ['class_second', 'class_third','sex_male','fare','alone','embark_town_southampton']
all_model_columns = ['fare', 'alone', 'embark_town_queenstown', 'embark_town_southampton',
                     'class_second', 'class_third', 'sex_male']

In [81]:
# get column combinations
column_combinations = []
for k in range(1, len(strong_model_columns)+1):
    for combo in list(itertools.combinations(strong_model_columns, k)):
        column_combinations.append(list(combo))
        
column_combinations

[['class_second'],
 ['class_third'],
 ['sex_male'],
 ['fare'],
 ['alone'],
 ['embark_town_southampton'],
 ['class_second', 'class_third'],
 ['class_second', 'sex_male'],
 ['class_second', 'fare'],
 ['class_second', 'alone'],
 ['class_second', 'embark_town_southampton'],
 ['class_third', 'sex_male'],
 ['class_third', 'fare'],
 ['class_third', 'alone'],
 ['class_third', 'embark_town_southampton'],
 ['sex_male', 'fare'],
 ['sex_male', 'alone'],
 ['sex_male', 'embark_town_southampton'],
 ['fare', 'alone'],
 ['fare', 'embark_town_southampton'],
 ['alone', 'embark_town_southampton'],
 ['class_second', 'class_third', 'sex_male'],
 ['class_second', 'class_third', 'fare'],
 ['class_second', 'class_third', 'alone'],
 ['class_second', 'class_third', 'embark_town_southampton'],
 ['class_second', 'sex_male', 'fare'],
 ['class_second', 'sex_male', 'alone'],
 ['class_second', 'sex_male', 'embark_town_southampton'],
 ['class_second', 'fare', 'alone'],
 ['class_second', 'fare', 'embark_town_southampton

In [82]:
def get_feature_combos(features):
    '''
    Returns a list of all possible feature combinations
    '''
    # get column combinations
    feature_combinations = []
    for k in range(1, len(features)+1):
        for combo in list(itertools.combinations(features, k)):
            feature_combinations.append(list(combo))
    return feature_combinations

In [92]:
def run_log_reg_model(train, validate, target, features, random_state=None):   
    '''
    takes in train set, validate set, target, columns to model
    gets all column combinations and runs decision tree models for each combination
    scores for each model are stored in a DataFrame, scores_df
    
    Returns scores_df, a DataFrame of model scores
    '''
    
    # get columns combination
    feature_combos = get_feature_combos(features)
    
    # for each combo of features
    for i, features in enumerate(feature_combos):
        
        # split X and y
        X_train, y_train = x_y_split(data_set=train, target=target, features=features)

        X_validate, y_validate = x_y_split(data_set=validate, target=target, features=features)
        
        scores = []
        
        # hyperparameter tuning
        for c in [1*10**x for x in range(-3, 4)]:
            # Make KNN classifier 
            clf = LogisticRegression(C=c, max_iter=150, random_state=123)

            # Fit KNN classifier
            clf.fit(X_train, y_train)

            # Predict
            y_pred_train = clf.predict(X_train)
            y_pred_val = clf.predict(X_validate)

            output = {
                'features':features,
                'model': f'LogReg: C={c}',
                'train_accuracy': metrics.accuracy_score(y_train, y_pred_train),
                'train_precision': metrics.precision_score(y_train, y_pred_train),
                'train_recall/TPR': metrics.recall_score(y_train, y_pred_train),
                'train_f1': metrics.f1_score(y_train, y_pred_train),
                'validate_accuracy':  metrics.accuracy_score(y_validate, y_pred_val),
                'validate_precision': metrics.precision_score(y_validate, y_pred_val),
                'validate_recall/TPR': metrics.recall_score(y_validate, y_pred_val),
                'validate_f1': metrics.f1_score(y_validate, y_pred_val),
            }
            scores.append(output)
            
        # initialize scores DataFrame for first iteration
        if i == 0:
            scores_df = pd.DataFrame(scores)
        # concat scores DataFrames for the rest of the iterations
        else:
            scores_df = pd.concat([scores_df, pd.DataFrame(scores)])
            
    return scores_df

In [93]:
scores_df = run_log_reg_model(train, validate, 'survived', strong_model_columns, random_state=20)

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Undefi

In [94]:
scores_df.sort_values(by='validate_accuracy', ascending=False)

,features,model,train_accuracy,train_precision,train_recall/TPR,train_f1,validate_accuracy,validate_precision,validate_recall/TPR,validate_f1
1,"[class_second, class_third, sex_male]",LogReg: C=0.01,0.794543,0.958678,0.485356,0.644444,0.764045,0.933333,0.411765,0.571429
1,"[class_third, sex_male, embark_town_southampton]",LogReg: C=0.01,0.794543,0.958678,0.485356,0.644444,0.764045,0.933333,0.411765,0.571429
1,"[class_second, class_third, sex_male, embark_t...",LogReg: C=0.01,0.794543,0.958678,0.485356,0.644444,0.764045,0.933333,0.411765,0.571429
1,"[class_third, sex_male, alone]",LogReg: C=0.01,0.794543,0.958678,0.485356,0.644444,0.764045,0.933333,0.411765,0.571429
1,"[class_third, sex_male]",LogReg: C=0.01,0.794543,0.958678,0.485356,0.644444,0.764045,0.933333,0.411765,0.571429
...,...,...,...,...,...,...,...,...,...,...
5,[alone],LogReg: C=100,0.621188,0.506024,0.527197,0.516393,0.606742,0.485294,0.485294,0.485294
6,[class_second],LogReg: C=1000,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424
5,[class_second],LogReg: C=100,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424
4,[class_second],LogReg: C=10,0.619583,0.507812,0.271967,0.354223,0.578652,0.387097,0.176471,0.242424


In [95]:
best_models = scores_df.sort_values(by='validate_accuracy', ascending=False).head(3)
best_models['model'].iloc[1]

'LogReg: C=0.01'

In [96]:
for i in range(2):
    row = best_models.iloc[i]
    print(f'''features: {row['features']}, : model:{row['model']}, accuracy: {round(row['validate_accuracy'], 3)}''')

features: ['class_second', 'class_third', 'sex_male'], : model:LogReg: C=0.01, accuracy: 0.764
features: ['class_third', 'sex_male', 'embark_town_southampton'], : model:LogReg: C=0.01, accuracy: 0.764
